# This is the capstone project notebook

In [1]:
import pandas as pd
import numpy as np



# Create a dataframe from wikipedia that will consist of three columns: PostalCode, Borough, and Neighborhood

In [2]:
# The code was removed by Watson Studio for sharing.

,Postal_Code,Borough,Neighborhood
0,MIJ,SCARBOROUGH,SCARBOROUGH VILLAGE
1,M9W,ETOBICOKE NORTHWEST,"CLAIRVILLE, HUMBERWOOD, WOODBINE DOWNS, WEST H..."
2,M9V,ETOBICOKE,"SOUTH STEELES, SILVERSTONE, HUMBERGATE, JAMEST..."
3,M9R,ETOBICOKE,"KINGVIEW VILLAGE, ST. PHILLIPS, MARTIN GROVE G..."
4,M9P,ETOBICOKE,WESTMOUNT


# Get the latitude and the longitude coordinates of each neighborhood by Postal Code. 

In [3]:
# add latitude and longitude

In [4]:

body = client_83a01ad55ed74e879ad36f9e780d4e6b.get_object(Bucket='financialmarketscustomerofferaffi-donotdelete-pr-0clbhikycqkxw6',Key='Geospatial_Coordinates1.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_5 = pd.read_csv(body)
df_data_5.head()


,Postal_Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


# Merge latitude and longitude dataframe by Postal Code to pair neighborhoods with their latitude and longitude

In [5]:
# Merge Latitude and Longitude dataframe by Postal Code to the neighborhoods and boroughs dataframe
df_data_4 = pd.merge(df_data_3, df_data_5, how='inner', on = 'Postal_Code')
df_data_4.head()

,Postal_Code,Borough,Neighborhood,Latitude,Longitude
0,M9W,ETOBICOKE NORTHWEST,"CLAIRVILLE, HUMBERWOOD, WOODBINE DOWNS, WEST H...",43.706748,-79.594054
1,M9V,ETOBICOKE,"SOUTH STEELES, SILVERSTONE, HUMBERGATE, JAMEST...",43.739416,-79.588437
2,M9R,ETOBICOKE,"KINGVIEW VILLAGE, ST. PHILLIPS, MARTIN GROVE G...",43.688905,-79.554724
3,M9P,ETOBICOKE,WESTMOUNT,43.696319,-79.532242
4,M9N,YORK,WESTON,43.706876,-79.518188


# Explore and cluster the neighborhoods in Toronto - there are 12 boroughs and 102 neighborhoods.

In [6]:
print('The merged dataframe has {} boroughs and {} neighborhoods in Toronto.'.format(
        len(df_data_4['Borough'].unique()),
        df_data_4.shape[0]
    )
)

The merged dataframe has 12 boroughs and 102 neighborhoods in Toronto.


In [7]:
# install map visualizations
!pip install folium
!pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
Libraries imported.


# Get the latitude and longitude of Toronto

In [8]:
# get latitude and longitude of Toronto from geopy
from geopy.geocoders import Nominatim

address = 'Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.6534817, -79.3839347.


# Display all Postal Codes in Toronto on a Map

In [9]:
# create map of Toronto using latitude and longitude values that plots all the Postal Codes
map_toronto = folium.Map(location=[43.6534817, -79.3839347], zoom_start=10)

# Add a marker for each postal code
for i in range(0,len(df_data_4)):
   folium.Marker(
      location=[df_data_4.iloc[i]['Latitude'], df_data_4.iloc[i]['Longitude']],
      popup=df_data_4.iloc[i]['Postal_Code'],
   ).add_to(map_toronto)

# Show the map 
map_toronto

# Explore First neighborhood

In [10]:
toronto_data = df_data_4.reset_index(drop=True)
toronto_data.head()

,Postal_Code,Borough,Neighborhood,Latitude,Longitude
0,M9W,ETOBICOKE NORTHWEST,"CLAIRVILLE, HUMBERWOOD, WOODBINE DOWNS, WEST H...",43.706748,-79.594054
1,M9V,ETOBICOKE,"SOUTH STEELES, SILVERSTONE, HUMBERGATE, JAMEST...",43.739416,-79.588437
2,M9R,ETOBICOKE,"KINGVIEW VILLAGE, ST. PHILLIPS, MARTIN GROVE G...",43.688905,-79.554724
3,M9P,ETOBICOKE,WESTMOUNT,43.696319,-79.532242
4,M9N,YORK,WESTON,43.706876,-79.518188


In [11]:
print('The city of Toronto has {} neighborhoods.'.format(
        len(toronto_data['Neighborhood'].unique()),
        toronto_data.shape[0]
    )
)

The city of Toronto has 102 neighborhoods.


In [12]:
toronto_data.loc[0, 'Neighborhood'] # explore the first neighborhood

'CLAIRVILLE, HUMBERWOOD, WOODBINE DOWNS, WEST HUMBER, KIPLING HEIGHTS, REXDALE, ELMS, TANDRIDGE, OLD REXDALE'

In [13]:
neighborhood_latitude = toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of CLAIRVILLE, HUMBERWOOD, WOODBINE DOWNS, WEST HUMBER, KIPLING HEIGHTS, REXDALE, ELMS, TANDRIDGE, OLD REXDALE are 43.706748299999994, -79.5940544.


In [14]:
# The code was removed by Watson Studio for sharing.

log in to Foursquare API


In [15]:
# limit of number of venues returned by Foursquare API
LIMIT = 100 
radius = 500

In [16]:
# request data from Foursquare API to find 100 venues within 500 meter radius of the neighborhood latitude and longitude
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius,
    LIMIT)
results = requests.get(url).json()

In [17]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [18]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


,name,categories,lat,lng
0,Economy Rent A Car,Rental Car Location,43.708471,-79.589943
1,Logistics Distribution,Bar,43.707554,-79.589252
2,Saand Rexdale,Drugstore,43.705072,-79.598725
3,PC Garden,Garden Center,43.706539,-79.599359
4,Vectra Heavy Haulers,Truck Stop,43.704891,-79.599410


In [19]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

5 venues were returned by Foursquare.


# Expand the venue search for all 102 neighborhoods in Toronto

In [20]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [21]:
df_data_4_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

CLAIRVILLE, HUMBERWOOD, WOODBINE DOWNS, WEST HUMBER, KIPLING HEIGHTS, REXDALE, ELMS, TANDRIDGE, OLD REXDALE
SOUTH STEELES, SILVERSTONE, HUMBERGATE, JAMESTOWN, MOUNT OLIVE, BEAUMOND HEIGHTS, THISTLETOWN, ALBION GARDENS
KINGVIEW VILLAGE, ST. PHILLIPS, MARTIN GROVE GARDENS, RICHVIEW GARDENS
WESTMOUNT
WESTON
HUMBERLEA, EMERY
HUMBER SUMMIT
ERINGATE, BLOORDALE GARDENS, OLD BURNHAMTHORPE, MARKLAND WOOD
WEST DEANE PARK, PRINCESS GARDENS, MARTIN GROVE, ISLINGTON, CLOVERDALE
ISLINGTON AVENUE
MIMICO NORTHWEST, QUEENSWAY WEST, SOUTH OF BLOOR, KINGSWAY PARK SOUTHWEST, ROYAL YORK SOUTHWEST
OLD MILL SOUTH, KING'S MILL PARK, SUNNYLEA, HUMBER BAY, MIMICO NORTHEAST, QUEENSWAY EAST, ROYAL YORK SOUTHEAST, KINGWAY PARK SOUTHEAST
KINGSWAY, MONTGOMERY ROAD, OLD MILL NORTH
ALDERWOOD, LONG BRANCH
NEW TORONTO, MIMICO SOUTH, HUMBER BAY SHORES
ENCLAVE OF M4L
ENCLAVE OF L4W
ONTARIO PROVINCIAL GOVERNMENT
RUNNYMEDE, SWANSEA
PARKDALE, RONCESVALLES
HIGH PARK, JUNCTION SOUTH
RUNNYMEDE, JUNCTION NORTH
DEL RAY, MOUNT DEN

In [22]:
# check the size of the dataframe
print(df_data_4_venues.shape)
df_data_4_venues.head()

(2153, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"CLAIRVILLE, HUMBERWOOD, WOODBINE DOWNS, WEST H...",43.706748,-79.594054,Economy Rent A Car,43.708471,-79.589943,Rental Car Location
1,"CLAIRVILLE, HUMBERWOOD, WOODBINE DOWNS, WEST H...",43.706748,-79.594054,Logistics Distribution,43.707554,-79.589252,Bar
2,"CLAIRVILLE, HUMBERWOOD, WOODBINE DOWNS, WEST H...",43.706748,-79.594054,Saand Rexdale,43.705072,-79.598725,Drugstore
3,"CLAIRVILLE, HUMBERWOOD, WOODBINE DOWNS, WEST H...",43.706748,-79.594054,PC Garden,43.706539,-79.599359,Garden Center
4,"CLAIRVILLE, HUMBERWOOD, WOODBINE DOWNS, WEST H...",43.706748,-79.594054,Vectra Heavy Haulers,43.704891,-79.599410,Truck Stop


# Count the number of venues per neighborhood

In [23]:
df_data_4_venues.groupby('Neighborhood').count() # number of venues that were returned for each neighborhood

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
AGINCOURT,5,5,5,5,5,5
"ALDERWOOD, LONG BRANCH",9,9,9,9,9,9
"BATHURST MANOR, WILSON HEIGHTS, DOWNSVIEW NORTH",23,23,23,23,23,23
BAYVIEW VILLAGE,4,4,4,4,4,4
"BEDFORD PARK, LAWRENCE MANOR EAST",24,24,24,24,24,24
...,...,...,...,...,...,...
WILLOWDALE WEST,7,7,7,7,7,7
"WILLOWDALE, NEWTONBROOK",1,1,1,1,1,1
WOBURN,3,3,3,3,3,3


# Map the categories via one hot encoding

In [24]:
# one hot encoding
df_data_4_onehot = pd.get_dummies(df_data_4_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
df_data_4_onehot['Neighborhood'] = df_data_4_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [df_data_4_onehot.columns[-1]] + list(df_data_4_onehot.columns[:-1])
df_data_4_onehot = df_data_4_onehot[fixed_columns]

df_data_4_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Trail,Train Station,Truck Stop,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [25]:
df_data_4_onehot.shape

(2153, 272)

In [26]:
# number of venues that were returned for each neighborhood
df_data_4_grouped = df_data_4_onehot.groupby('Neighborhood').mean().reset_index()
df_data_4_grouped

df_data_4_grouped.shape

(98, 272)

# List each of the top 5 most commons venues per neighborhood

In [27]:
# print each neighborhood with the 5 most common venues from Foursquare API
num_top_venues = 5

for hood in df_data_4_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = df_data_4_grouped[df_data_4_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----AGINCOURT----
                       venue  freq
0               Skating Rink   0.2
1                     Lounge   0.2
2             Breakfast Spot   0.2
3             Clothing Store   0.2
4  Latin American Restaurant   0.2


----ALDERWOOD, LONG BRANCH----
            venue  freq
0     Pizza Place  0.22
1             Pub  0.11
2        Pharmacy  0.11
3  Sandwich Place  0.11
4             Gym  0.11


----BATHURST MANOR, WILSON HEIGHTS, DOWNSVIEW NORTH----
                       venue  freq
0                       Bank  0.09
1                Coffee Shop  0.09
2                 Restaurant  0.04
3  Middle Eastern Restaurant  0.04
4         Chinese Restaurant  0.04


----BAYVIEW VILLAGE----
                      venue  freq
0                      Café  0.25
1        Chinese Restaurant  0.25
2                      Bank  0.25
3       Japanese Restaurant  0.25
4  Mediterranean Restaurant  0.00


----BEDFORD PARK, LAWRENCE MANOR EAST----
                venue  freq
0  Italian Restaurant  0.

# Neighborhood Downsview identified as having high percentage of food trucks

In [28]:
# sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

# Display the top 10 venues overall for each Toronto neighborhood

In [29]:
# display top 10 venues overall for each  neighborhood in a dataframe

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = df_data_4_grouped['Neighborhood']

for ind in np.arange(df_data_4_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(df_data_4_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,AGINCOURT,Lounge,Latin American Restaurant,Skating Rink,Clothing Store,Breakfast Spot,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run
1,"ALDERWOOD, LONG BRANCH",Pizza Place,Pub,Coffee Shop,Gym,Pharmacy,Sandwich Place,Playground,Dance Studio,Women's Store,Department Store
2,"BATHURST MANOR, WILSON HEIGHTS, DOWNSVIEW NORTH",Bank,Coffee Shop,Ice Cream Shop,Chinese Restaurant,Supermarket,Restaurant,Sushi Restaurant,Middle Eastern Restaurant,Shopping Mall,Deli / Bodega
3,BAYVIEW VILLAGE,Café,Japanese Restaurant,Bank,Chinese Restaurant,Women's Store,Dim Sum Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant
4,"BEDFORD PARK, LAWRENCE MANOR EAST",Coffee Shop,Italian Restaurant,Sandwich Place,Liquor Store,Restaurant,Indian Restaurant,Café,Pub,Sushi Restaurant,Breakfast Spot
...,...,...,...,...,...,...,...,...,...,...,...
93,WILLOWDALE WEST,Coffee Shop,Pizza Place,Discount Store,Supermarket,Butcher,Pharmacy,Grocery Store,Department Store,Electronics Store,Eastern European Restaurant
94,"WILLOWDALE, NEWTONBROOK",Park,Women's Store,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore
95,WOBURN,Coffee Shop,Korean BBQ Restaurant,Women's Store,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Dumpling Restaurant
96,WOODBINE HEIGHTS,Park,Spa,Beer Store,Skating Rink,Athletics & Sports,Intersection,Curling Ice,Women's Store,Dog Run,Discount Store


# Cluster the 102 Toronto neighborhoods into clusters to identify opportunities of similar neighborhoods 

In [30]:
# cluster the 102 Toronto neighborhoods into clusters
kclusters = 8

df_data_4_grouped_clustering = df_data_4_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df_data_4_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 6, 0], dtype=int32)

In [31]:
# add clustering labels to create a dataframe that includes the clusters and top 10 venues per neighborhood
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge grouped data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postal_Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M9W,ETOBICOKE NORTHWEST,"CLAIRVILLE, HUMBERWOOD, WOODBINE DOWNS, WEST H...",43.706748,-79.594054,0.0,Bar,Rental Car Location,Garden Center,Drugstore,Truck Stop,Women's Store,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
1,M9V,ETOBICOKE,"SOUTH STEELES, SILVERSTONE, HUMBERGATE, JAMEST...",43.739416,-79.588437,0.0,Grocery Store,Coffee Shop,Pharmacy,Pizza Place,Beer Store,Sandwich Place,Fast Food Restaurant,Fried Chicken Joint,Distribution Center,Dessert Shop
2,M9R,ETOBICOKE,"KINGVIEW VILLAGE, ST. PHILLIPS, MARTIN GROVE G...",43.688905,-79.554724,0.0,Park,Sandwich Place,Bus Line,Mobile Phone Shop,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Ethiopian Restaurant,Distribution Center
3,M9P,ETOBICOKE,WESTMOUNT,43.696319,-79.532242,0.0,Pizza Place,Intersection,Coffee Shop,Middle Eastern Restaurant,Sandwich Place,Playground,Discount Store,Chinese Restaurant,Women's Store,Distribution Center
4,M9N,YORK,WESTON,43.706876,-79.518188,1.0,Jewelry Store,Women's Store,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Falafel Restaurant


# Evaluate neighborhood clusters to identify best opportunities for expanding food truck business based on similarity to Downsview.

In [33]:
# identify the venue cluster types that distinguish the first cluster
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[2] + list(range(4, toronto_merged.shape[1]))]]

,Neighborhood,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"CLAIRVILLE, HUMBERWOOD, WOODBINE DOWNS, WEST H...",-79.594054,0.0,Bar,Rental Car Location,Garden Center,Drugstore,Truck Stop,Women's Store,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
1,"SOUTH STEELES, SILVERSTONE, HUMBERGATE, JAMEST...",-79.588437,0.0,Grocery Store,Coffee Shop,Pharmacy,Pizza Place,Beer Store,Sandwich Place,Fast Food Restaurant,Fried Chicken Joint,Distribution Center,Dessert Shop
2,"KINGVIEW VILLAGE, ST. PHILLIPS, MARTIN GROVE G...",-79.554724,0.0,Park,Sandwich Place,Bus Line,Mobile Phone Shop,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Ethiopian Restaurant,Distribution Center
3,WESTMOUNT,-79.532242,0.0,Pizza Place,Intersection,Coffee Shop,Middle Eastern Restaurant,Sandwich Place,Playground,Discount Store,Chinese Restaurant,Women's Store,Distribution Center
6,HUMBER SUMMIT,-79.565963,0.0,Gym,Pizza Place,Women's Store,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,"GOLDEN MILE, CLAIRLEA, OAKRIDGE",-79.284577,0.0,Bakery,Bus Line,Soccer Field,Park,Ice Cream Shop,Intersection,Bus Station,Dog Run,Discount Store,Distribution Center
96,"KENNEDY PARK, IONVIEW, EAST BIRCHMONT PARK",-79.262029,0.0,Department Store,Chinese Restaurant,Bus Station,Coffee Shop,Discount Store,Hobby Shop,Diner,Distribution Center,Dog Run,Doner Restaurant
97,CEDARBRAE,-79.239476,0.0,Fried Chicken Joint,Bakery,Hakka Restaurant,Athletics & Sports,Bank,Thai Restaurant,Gas Station,Caribbean Restaurant,Distribution Center,Diner
98,WOBURN,-79.216917,0.0,Coffee Shop,Korean BBQ Restaurant,Women's Store,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Dumpling Restaurant


In [34]:
# identify the venue cluster types that distinguish cluster 2
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[2] + list(range(4, toronto_merged.shape[1]))]]

,Neighborhood,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,WESTON,-79.518188,1.0,Jewelry Store,Women's Store,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Falafel Restaurant
38,"FOREST HILL NORTH, FOREST HILL WEST",-79.411307,1.0,Park,Jewelry Store,Sushi Restaurant,Trail,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Electronics Store,Department Store


In [35]:
# identify the venue cluster types that distinguish cluster 3
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[2] + list(range(4, toronto_merged.shape[1]))]]

,Neighborhood,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
81,"WILLOWDALE, NEWTONBROOK",-79.408493,2.0,Park,Women's Store,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore


In [36]:
# identify the venue cluster types that distinguish cluster 4
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[2] + list(range(4, toronto_merged.shape[1]))]]

,Neighborhood,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
101,"MALVERN, ROUGE",-79.194353,3.0,Fast Food Restaurant,Women's Store,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore


In [37]:
# identify the venue cluster types that distinguish cluster 5
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[2] + list(range(4, toronto_merged.shape[1]))]]

,Neighborhood,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,"WEST DEANE PARK, PRINCESS GARDENS, MARTIN GROV...",-79.554724,4.0,Bakery,Women's Store,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Falafel Restaurant


In [38]:
# identify the venue cluster types that distinguis cluster 6
toronto_merged.loc[toronto_merged['Cluster Labels'] == 5, toronto_merged.columns[[2] + list(range(4, toronto_merged.shape[1]))]]

,Neighborhood,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,"HUMBERLEA, EMERY",-79.532242,5.0,Baseball Field,Furniture / Home Store,Women's Store,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
11,"OLD MILL SOUTH, KING'S MILL PARK, SUNNYLEA, HU...",-79.498509,5.0,Baseball Field,Construction & Landscaping,Women's Store,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore
70,DOWNSVIEW,-79.495697,5.0,Baseball Field,Food Truck,Women's Store,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dessert Shop


In [39]:
# identify the venue cluster types that distinguis cluster 7
toronto_merged.loc[toronto_merged['Cluster Labels'] == 6, toronto_merged.columns[[2] + list(range(4, toronto_merged.shape[1]))]]

,Neighborhood,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
28,CALEDONIA-FAIRBANKS,-79.453512,6.0,Park,Women's Store,Pool,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run
52,ROSEDALE,-79.377529,6.0,Park,Playground,Trail,Electronics Store,Eastern European Restaurant,Escape Room,Dumpling Restaurant,Drugstore,Donut Shop,Deli / Bodega
54,"MOORE PARK, SUMMERHILL EAST",-79.383160,6.0,Park,Summer Camp,Playground,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
62,DANFORTH EAST,-79.338106,6.0,Park,Convenience Store,Intersection,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Women's Store
72,"DOWNSVIEW EAST, CFB TORONTO",-79.464763,6.0,Park,Airport,Playground,Women's Store,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
79,YORK MILLS WEST,-79.400049,6.0,Park,Convenience Store,Construction & Landscaping,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
88,"MILLIKEN, AGINCOURT NORTH, STEELES EAST, L'AMO...",-79.284577,6.0,Park,Intersection,Playground,Women's Store,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run


# The neighborhood (DOWNSVIEW) with 1st most common venue = Baseball Field has 2nd most common venue = Food Truck. Find best opportunity to locate a second food truck to maximize profit.

# Neighborhoods with Baseball Field as 1st most common venue without Food Trucks in top 10 common venues listing are in North York (Humberlea & Emery) and Etobicoke (Old Mill South, King's Mill Park, Sunnylea, Humber Bay, Mimico Northeast, Queensway East, Royal York Southeast, & Kingway Park Southeast). Focusing on these two neighborhoods to find ideal location for second food truck for best profit potential. Map created to identify neighborhoods. 

In [40]:
# create map of Toronto to display the locations of the two alternate neighborhoods compared to Food Truck Central
map_toronto = folium.Map(location=[43.6534817, -79.3839347], zoom_start=11)

# Add a marker for each postal code
folium.Marker(location=[43.6362579, -79.4985091],icon= folium.Icon(color='green'), popup='Etobicoke Neighborhoods',tooltip='Click here to see Popup').add_to(map_toronto)
folium.Marker(location=[43.7247659, -79.5322424],popup='<strong>North York Neighborhoods</strong>',tooltip='<strong>Click here to see Popup</strong>').add_to(map_toronto)
folium.Marker(location=[43.7284964, -79.4956974],icon= folium.Icon(color='red'), popup='<h3 style="color:green;">Downsview Food Truck Central</h3>',tooltip='<strong>Click here to see Popup</strong>').add_to(map_toronto)

# Show the map 
map_toronto

# Query Foursquare API for Baseball fields in the Etobicoke area most similar to Downsview to identify good location for food truck expansion

In [45]:
# The code was removed by Watson Studio for sharing.

{'meta': {'code': 200, 'requestId': '6100e43924aae1771b97e31f'},
 'response': {'suggestedRadius': 3097,
  'headerLocation': 'Toronto',
  'headerFullLocation': 'Toronto',
  'headerLocationGranularity': 'city',
  'query': 'baseball field',
  'totalResults': 13,
  'suggestedBounds': {'ne': {'lat': 43.69139456253491,
    'lng': -79.39816144108772},
   'sw': {'lat': 43.59828589475782, 'lng': -79.55747875571251}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4ae33298f964a520909121e3',
       'name': 'Sorauren Avenue Park',
       'contact': {},
       'location': {'address': '289 Sorauren Ave.',
        'crossStreet': 'at Wabash Ave.',
        'lat': 43.648443306627975,
        'lng': -79.44340467453002,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.64